In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines:  model upload, predict, and evaluate using google-cloud-pipeline-components

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_upload_predict_evaluate.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fpipelines%2Fgoogle_cloud_pipeline_components_model_upload_predict_evaluate.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_upload_predict_evaluate.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br>
      Open in Vertex AI Workbench
     </a>
  </td> 
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/google_cloud_pipeline_components_model_upload_predict_evaluate.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br>
      View on GitHub
    </a>
  </td>
</table>

*Note: This notebook uses KFP 1.x and GCPC 1.x. We recommend using 2.x*

## Overview

This notebook shows how to use the components defined in [`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) in conjunction with an experimental `evaluation` method, to build a [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines) workflow that uploads a tabular custom model as a model resource, creates a BatchPredictionJob resource, and evaluates the model resource with the BatchPredictionJob results to create an evaluation `system.Metrics` artifact. 

Learn more about [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) and [Vertex AI Model components](https://cloud.google.com/vertex-ai/docs/pipelines/model-endpoint-component).

### Objective

In this tutorial, you learn how to evaluate a custom model using a pipeline with components from `google_cloud_pipeline_components` and a custom pipeline component you build.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Pipelines
- Vertex AI Model Registry
- Vertex AI Batch Prediction

The steps performed include:

- Upload a pretrained model as a model resource.
- Run a BatchPredictionJob on the model resource with ground truth data.
- Generate evaluation Metrics artifact about the model resource.
- Compare the evaluation metrics to a threshold.


### Dataset

The dataset used for this tutorial is part of the [safe driver prediction Kaggle competition](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/overview). The model has been trained on this data, and ground truth is used for evaluation.

The dataset predicts the whether or not a claim was filed for the policy holder.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform 'google-cloud-pipeline-components<2' 'kfp<2' google-api-python-client 'protobuf==3.20.3'

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

- *{Note to notebook author: For any user-provided strings that need to be unique (like bucket names or model ID's), append "-unique" to the end so proper testing can occur}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### Initialize Vertex AI SDK for Python

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI, location=LOCATION)

#### UUID

If you're in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Service Account

You use a service account to create Vertex AI Pipeline jobs.

If you do not want to use your project's Compute Engine service account, set `SERVICE_ACCOUNT` to another service account ID.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = ! gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Import libraries and define constants

In [ ]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components.experimental.evaluation import \
    ModelEvaluationOp as evaluation_op
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components.v1.batch_predict_job import \
    ModelBatchPredictOp as batch_prediction_op
from google_cloud_pipeline_components.v1.model import \
    ModelUploadOp as model_upload_op
from kfp.v2 import compiler
from kfp.v2.components import importer_node
from kfp.v2.dsl import Input, Metrics, component

Setup up the following constant for Vertex AI Pipelines:

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/safe_driver".format(BUCKET_URI)

## Create component for comparing evalution metrics to a threshold

First, you create your own component that takes the evaluation metrics artifact as input, checks the threshold and return yes/no decision. It is used in a subsequent dsl.Condition() to decide whether the model should proceed to the next step i.e., online deployment.

The component takes the following parameters:

- `eval_metrics`: The evaluation metrics artifact returned from `ModelEvaluation` component.
- `metric_name`: The key name for the metric entry to make the comparison to.
- `threshold`: The threshold for the metric value for a yes/no decision.

In [ ]:
@component()
def compare(eval_metrics: Input[Metrics], metric_name: str, threshold: float) -> str:
    path = eval_metrics.path
    # print("PATH", path)

    gs_prefix = "gs://"
    gcsfuse_prefix = "/gcs/"
    if path.startswith(gs_prefix):
        path = path.replace(gs_prefix, gcsfuse_prefix)

    import json

    with open(path) as f:
        data = json.load(f)

    slices = data["slicedMetrics"]

    metrics = slices[0]["metrics"]["classification"]
    # print("METRIC KEYS", metrics.keys())

    value = metrics[metric_name]
    if value > threshold:
        return "true"

    return "false"

## Define evaluation pipeline that uses components from `google_cloud_pipeline_components`

Next, define the pipeline.

[`google_cloud_pipeline_components`](https://github.com/kubeflow/pipelines/tree/master/components/google-cloud) components used to define the pipeline are: upload the model, run batch prediction, and evaluate the model with the given predictions.

View the definition of the [upload model component](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/aiplatform/model/upload_model/component.yaml).

View the definition of the [batch prediction component](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/aiplatform/batch_predict_job/component.yaml).

View the definition of the [experimental evaluation component](https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/experimental/evaluation/component.yaml).

In [ ]:
DATA_URIS = [
    "gs://cloud-samples-data/vertex-ai/dataset-management/datasets/safe_driver/dataset_safe_driver_train_10k.csv"
]
MODEL_URI = "gs://cloud-samples-data/vertex-ai/google-cloud-aiplatform-ci-artifacts/models/safe_driver/model"
# Create working dir
WORKING_DIR = f"{PIPELINE_ROOT}/{UUID}"
MODEL_DISPLAY_NAME = f"safe-driver-{UUID}"
BATCH_PREDICTION_DISPLAY_NAME = f"batch-prediction-on-pipelines-model-{UUID}"


@kfp.dsl.pipeline(name="upload-evaluate-" + UUID)
def pipeline(
    metric: str,
    threshold: float,
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    batch_prediction_display_name: str = BATCH_PREDICTION_DISPLAY_NAME,
    batch_prediction_data_uris: list = DATA_URIS,
):
    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=MODEL_URI,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "artifactUri": MODEL_URI,
            "predictSchemata": {
                "predictionSchemaUri": MODEL_URI + "/prediction_schema.yaml",
                "instanceSchemaUri": MODEL_URI + "/instance.yaml",
            },
            "containerSpec": {
                "imageUri": "us-docker.pkg.dev/vertex-ai/automl-tabular/prediction-server:prod",
                "healthRoute": "/health",
                "predictRoute": "/predict",
            },
        },
    )

    model_task = model_upload_op(
        project=project,
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    )

    batch_prediction_task = batch_prediction_op(
        project=project,
        model=model_task.outputs["model"],
        job_display_name=batch_prediction_display_name,
        gcs_source_uris=batch_prediction_data_uris,
        instances_format="csv",
        predictions_format="jsonl",
        gcs_destination_output_uri_prefix=WORKING_DIR,
        machine_type="n1-standard-32",
    )

    eval_task = evaluation_op(
        project=project,
        root_dir=WORKING_DIR,
        problem_type="classification",
        classification_type="multiclass",
        ground_truth_column="target",
        class_names=["0", "1"],
        predictions_format="jsonl",
        batch_prediction_job=batch_prediction_task.outputs["batchpredictionjob"],
    )

    _ = compare(
        eval_metrics=eval_task.outputs["evaluation_metrics"],
        metric_name=metric,
        threshold=threshold,
    )

## Compile the pipeline

Next, compile the pipeline.

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="evaluation_demo_pipeline.json",
)

## Run the pipeline

Next, run the pipeline.

In [ ]:
DISPLAY_NAME = "safe_driver" + UUID

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="evaluation_demo_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"metric": "auPrc", "threshold": 0.95},
    enable_caching=True,
)

job.run()

! rm evaluation_demo_pipeline.json

Click on the generated link to see your run in the Cloud Console.

In the UI, the nodes of pipeline DAG expand or collapse when you click on them.

### View the pipeline results and evaluation metrics

In [ ]:
PROJECT_NUMBER = job.gca_resource.name.split("/")[1]
print(PROJECT_NUMBER)


def print_pipeline_output(job, output_task_name):
    JOB_ID = job.name
    print(JOB_ID)
    artifact = ""
    for _ in range(len(job.gca_resource.job_detail.task_details)):
        TASK_ID = job.gca_resource.job_detail.task_details[_].task_id
        EXECUTE_OUTPUT = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/executor_output.json"
        )
        GCP_RESOURCES = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/gcp_resources"
        )
        EVALUATION_METRICS = (
            PIPELINE_ROOT
            + "/"
            + PROJECT_NUMBER
            + "/"
            + JOB_ID
            + "/"
            + output_task_name
            + "_"
            + str(TASK_ID)
            + "/evaluation_metrics"
        )
        # Check if file exists, 0 is success
        !gsutil -q stat $EXECUTE_OUTPUT
        if _exit_code == 0:
            ! gsutil cat $EXECUTE_OUTPUT
            artifact = EXECUTE_OUTPUT
            break
        !gsutil -q stat $GCP_RESOURCES
        if _exit_code == 0:
            ! gsutil cat $GCP_RESOURCES
            artifact = GCP_RESOURCES
            break
        !gsutil -q stat $EVALUATION_METRICS
        if _exit_code == 0:
            ! gsutil cat $EVALUATION_METRICS
            artifact = EVALUATION_METRICS
            break

    return artifact


print("model-upload")
artifacts = print_pipeline_output(job, "model-upload")
print("\n\n")
print("model-batch-predict")
artifacts = print_pipeline_output(job, "model-batch-predict")
print("\n\n")
print("model-evaluation")
metrics = print_pipeline_output(job, "model-evaluation")
print("\n\n")
print("compare")
artifacts = print_pipeline_output(job, "compare")

### Delete pipeline job

After a pipeline job is completed, you can delete the pipeline job with the method `delete()`.  Prior to completion, a pipeline job can be canceled with the method `cancel()`.

In [ ]:
job.delete()

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

- Model
- Batch Job
- Cloud Storage Bucket (Set `delete_bucket` to **True** to delete the Cloud Storage bucket).

In [ ]:
import os

delete_bucket = False

# Delete the created model
models = aiplatform.Model.list(
    filter=f"display_name={MODEL_DISPLAY_NAME}", order_by="create_time"
)
if len(models) > 0:
    model = models[0]
    model.delete()
    print("Deleted model:", model)

# Delete the created batch-prediction job
batch_predictions = aiplatform.BatchPredictionJob.list(
    filter=f"display_name={BATCH_PREDICTION_DISPLAY_NAME}",
    order_by="create_time",
)
if len(batch_predictions) > 0:
    batch_prediction = batch_predictions[0]
    batch_prediction.delete()
    print("Deleted batch prediction job:", batch_prediction)

# Delete the Cloud Storage bucket
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI